Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

installing components

In [1]:
!conda install beautifulsoup4 -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.2       |           py36_0         162 KB
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2020.4.5.1         |           py36_0         155 KB
    openssl-1.1.1f             |       h7b6447c_0         2.5 MB
    soupsieve-2.0              |             py_0          33 KB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     pkgs/main/linux-64::beautifulsoup4-4.8.2-py36_0
  soupsieve          pkgs/main/noarch::soupsieve-2.0-py_0

The following packages will be SUPERSEDED by a higher-priority channel

In [2]:
!conda install lxml -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-4.5.0                 |   py36hefd8a0e_0         1.4 MB
    ------------------------------------------------------------
                                           Total:         1.8 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               pkgs/main/linux-64::lxml-4.5.0-py36hefd8a0e_0



lxml-4.5.0           | 1.4 MB    | ##################################### | 100% 
libxslt-1.1.33       | 426 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install requests -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    asn1crypto-1.3.0           |           py36_0         164 KB
    chardet-3.0.4              |        py36_1003         180 KB
    cryptography-2.8           |   py36h1ba5d50_0         552 KB
    idna-2.9                   |             py_1          49 KB
    pyopenssl-19.1.0           |           py36_0          87 KB
    pysocks-1.7.1              |           py36_0          30 KB
    requests-2.23.0            |           py36_0          91 KB
    urllib3-1.25.8             |           py36_0         169 KB
    ------------------------------------------------------------
                                           Total:         1.3 MB

The following NEW packages will be INSTALLED:

  asn1cry

In [4]:
from bs4 import BeautifulSoup
import requests

saving wikipedia page into soup variable for parsing

In [6]:
import urllib.request
with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [99]:
#table = soup.find('table', class_= 'wikitable')
columnnames = []
table = soup.table
columns = table.findAll('th')
header = []
for column in columns:
    header.append(column.text.rstrip("\n"))
header

['Postal code', 'Borough', 'Neighborhood']

making dataframe from table contents

In [100]:
import pandas as pd
#new_table = pd.DataFrame(columns=range(0,2), index = [0]) # I know the size 
   
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text.rstrip("\n"))
    output_rows.append(output_row)
output_rows

df = pd.DataFrame(output_rows)
df.shape

(181, 3)

In [101]:
df.head()

,0,1,2
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [102]:
df.columns =  header
df.drop([0], inplace=True)


In [103]:
df.head()

,Postal code,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
removing cells with Not assigned from the dataframe

In [104]:
newdf = df.drop(df[df.Borough == 'Not assigned'].index)
newdf


,Postal code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
161,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,Business reply mail Processing CentrE
170,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table. 


In [118]:
newdf.Neighborhood.replace(' /', ',', regex=True, inplace=True)
newdf.reset_index(inplace=True)
newdf

,level_0,index,Postal code,Borough,Neighborhood
0,0,3,M3A,North York,Parkwoods
1,1,4,M4A,North York,Victoria Village
2,2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...,...
98,98,161,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North"
99,99,166,M4Y,Downtown Toronto,Church and Wellesley
100,100,169,M7Y,East Toronto,Business reply mail Processing CentrE
101,101,170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [86]:
countempty = newdf.Neighborhood == 'Not assigned'

In [87]:
countempty.count()

103

there are now neighbourhoods with not assigned values

In [90]:
newdf.head()


,Postal code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [91]:
newdf.shape

(103, 3)